In [1]:
!pip install tensorflow pillow googletrans==4.0.0-rc1 gTTS ipywidgets

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.2 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=566f5c29d5dde49b57301340db4ab583bfbccbb4ce714c9eb24a947012ef11f8
  Stored in directory: /root/.cache/pip/wheels/95/0f/04/b17a72024b56a60e499ce1a6313d283ed5ba33

In [8]:
import tensorflow as tf
import pandas as pd
import numpy as np
from googletrans import Translator
from PIL import Image
import matplotlib.pyplot as plt
from gtts import gTTS
import IPython.display as ipd
import ipywidgets as widgets

In [21]:
model = tf.keras.applications.MobileNetV2(weights = 'imagenet')
decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

translator = Translator()

uploader = widgets.FileUpload(accept = 'image/*', multiple = False)
display(uploader)

def classify_and_translate(file, target_lang = 'es'):
  img = Image.open(file).resize((224,224))
  img_array = np.array(img)
  if img_array.shape[-1] == 4:
    img_array = img_array[..., :3]
  image_batch = np.expand_dims(img_array, axis = 0)
  image_batch = preprocess_input(image_batch)
  #Predict
  preds = model.predict(image_batch)
  decoded = decode_predictions(preds, top = 1)[0][0]
  label = decoded[1]
  confidence = decoded[2]
  print(f'Prediction: {label} ({confidence:.2f} confidence)')
  #Translate
  translated = translator.translate(label, dest = target_lang).text
  print(f'Translated to {target_lang}: {translated}')
  #Speak
  tts = gTTS(translated, lang = target_lang)
  tts.save('output.mp3')
  return translated, label, 'output.mp3'

def on_upload_change(change):
  for name, file_info in uploader.value.items():
    with open(name, 'wb') as f:
      f.write(file_info['content'])
    label, translated, audio_file = classify_and_translate(name, target_lang = 'bn')
    ipd.display(ipd.Audio(audio_file))
uploader.observe(on_upload_change, names = 'value')

FileUpload(value={}, accept='image/*', description='Upload')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Prediction: bell_pepper (0.20 confidence)
Translated to bn: বেল_মরিচ


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Prediction: puck (0.14 confidence)
Translated to bn: পাক


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Prediction: pomegranate (0.30 confidence)
Translated to bn: ডালিম


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Prediction: bell_pepper (0.23 confidence)
Translated to bn: বেল_মরিচ


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Prediction: monastery (0.48 confidence)
Translated to bn: মঠ


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Prediction: scoreboard (0.32 confidence)
Translated to bn: স্কোরবোর্ড


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Prediction: plate_rack (0.63 confidence)
Translated to bn: প্লেট_র্যাক
